In [1]:
import numpy as np
import pandas as pd
import random as rnd

In [2]:
data = pd.read_csv("C:\\Users\\elad\\Desktop\\Kaggle\\Load prediction\\train.csv")
data = np.array(data)
np.random.shuffle(data)

In [3]:
# Data completion
dataSize = data.shape[0]

for i in range(dataSize):
    if data[i,12] == 'N':
        data[i,12] = 0
    elif data[i,12] == 'Y':
        data[i,12] = 1
    else:
        print('WARNING IN DATA\'s CLASS')


# Co Applicant Income
counter = 0
for x in data[:,7]:
    if x > 0:
        counter += 1

medianCoApplicantIncome = int(np.sum(data[:,7]) / counter) + 0.0

for i in range(len(data[:,7])):
    if data[i,7] == 0:
        data[i,7] = medianCoApplicantIncome

        
# Loan Amount
counter = 0
sum = 0
for x in data[:,8]:
    if not np.isnan(x):
        counter += 1
        sum += x

medianLoanAmount = int(sum / counter) + 0.0
for i in range(len(data[:,8])):
    if np.isnan(data[i,8]):
        data[i,8] = medianLoanAmount
        

In [4]:
# Cleaning Data Calculations
maxApplicantIncome = np.max(data[:,6])
minApplicantIncome = np.min(data[:,6])
avgApplicantIncome = np.mean(data[:,6])
maxCoApplicantIncome = np.max(data[:,7])
minCoApplicantIncome = np.min(data[:,7])
avgCoApplicantIncome = np.mean(data[:,7])
maxLoanAmount = np.max(data[:,8])
minLoanAmount = np.min(data[:,8])
avgLoadAmount = np.mean(data[:,8])

def clean_sample(sample):
    #Gender
    if sample[1] == 'Male':
        sample[1] = 0
    elif sample[1] == 'Female':
        sample[1] = 1
    else:
        sample[1] = 0
        if rnd.randint(1,5) == 1:
            sample[1] = 1
            
    
    #Married
    if sample[2] == 'No':
        sample[2] = 0
    elif sample[2] == 'Yes':
        sample[2] = 1
    else:
        sample[2] = rnd.randint(0,1)
      
    #Dependents
    if sample[3] == '0':
        sample[3] = 0.0
    elif sample[3] == '1':
        sample[3] = 1.0
    elif sample[3] == '2':
        sample[3] = 2.0
    elif sample[3] == '3+':
        sample[3] = 3.0
    else:
        if rnd.randint(1,4) != 1:
            sample[3] = 0 
        else:
            sample[3] = rnd.randint(1,3)
    
    #Graduated
    if sample[4] == 'Not Graduate':
        sample[4] = 0
    elif sample[4] == 'Graduate':
        sample[4] = 1
    else:
        sample[4] = rnd.randint(0,1)
        
    #Self-Employed
    if sample[5] == 'No':
        sample[5] = 0
    elif sample[5] == 'Yes':
        sample[5] = 1
    else:
        sample[5] = 0
        if rnd.randint(1,4) == 1:
            sample[5] = 1
        
    #Applicant Income
    sample[6] = (sample[6] - avgApplicantIncome) / (maxApplicantIncome - minApplicantIncome)
    
    #Co Applicant Income
    sample[7] = (sample[7] - avgCoApplicantIncome) / (maxCoApplicantIncome - minCoApplicantIncome)
    
    # Loan Amount
    sample[8] = (sample[8] - avgLoadAmount) / (maxLoanAmount - minLoanAmount)
    
    # Loan Amount Term
    if sample[9] <= 120:
        sample[9] = 0
    elif sample[9] == 180 or sample[9] == 240:
        sample[9] = 1
    elif sample[9] == 360:
        sample[9] = 2
    elif sample[9] == 480:
        sample[9] = 3
    elif sample[9] == 300:
        sample[9] = 4
    else:
        sample[9] = 2
        if rnd.randint(1,4) == 1:
             sample[9] = [0,1,3,4][rnd.randint(0,3)]
        
   
    # Credit History
    if np.isnan(sample[10]):
        sample[10] = 1
        if len(sample) == 13:
            sample[10] = sample[12]
        else:
            if rnd.randint(1,5) == 1:
                sample[10] = 0
            
    # Property Area
    if sample[11] == 'Semiurban':
        sample[11] = 0
    elif sample[11] == 'Urban':
        sample[11] = 1
    elif sample[11] == 'Rural':
        sample[11] = 2
    else:
        print("ERROR IN PROPERY HISTORY")
    
    return sample

In [5]:
for x in data:
    clean_sample(x)
data = np.delete(data,3,1)

In [6]:
n = 500
f_n = 11
X_train = data[:n,1:f_n]
Y_train = np.array(data[:n,f_n], dtype='int')
X_test = data[n:, 1:f_n]
Y_test = np.array(data[n:,f_n], dtype='int')

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=100, max_depth=10,criterion='gini',max_features=(f_n-1))
random_forest.fit(X_train, Y_train)
acc_random_forest = random_forest.score(X_train, Y_train)
print("Random Forest Train accuracy :  " + str(acc_random_forest))
acc_random_forest = random_forest.score(X_test, Y_test)
print("Random Forest Test accuracy :  " + str(acc_random_forest))
random_forest.feature_importances_

Random Forest Train accuracy :  0.934
Random Forest Test accuracy :  0.7631578947368421


array([0.00940545, 0.02353752, 0.01975353, 0.01298977, 0.16114701,
       0.1110257 , 0.1357487 , 0.03639785, 0.4544761 , 0.03551837])

In [9]:
test = pd.read_csv("C:\\Users\\elad\\Desktop\\Kaggle\\Load prediction\\test.csv")
test = np.array(test)

In [10]:
# Data completion

for i in range(len(test[:,7])):
    if test[i,7] == 0:
        test[i,7] = medianCoApplicantIncome


for i in range(len(test[:,8])):
    if np.isnan(test[i,8]):
        test[i,8] = medianLoanAmount
        

In [12]:
np.shape(test)

(367, 12)

In [13]:
for x in test:
    clean_sample(x)
test = np.delete(test,3,1)

In [14]:
np.shape(test)

(367, 11)

In [246]:
pd.DataFrame(test[:10])

,0,1,2,3,4,5,6,7,8,9,10
0,LP001015,0,1,1,0,0.00391516,-2.57765e-06,-0.0526735,2,1,1
1,LP001022,0,1,1,0,-0.0287874,-0.0340715,-0.0295187,2,1,1
2,LP001031,0,1,1,0,-0.00499022,-0.0268688,0.0891499,2,1,1
3,LP001035,0,1,1,0,-0.0378907,-0.00895797,-0.0671453,2,1,1
4,LP001051,0,0,0,0,-0.0263137,-2.57765e-06,-0.0989832,2,1,1
5,LP001054,0,1,0,1,-0.0400552,0.012074,0.00810797,2,1,1
6,LP001055,1,0,0,0,-0.0393007,-2.57765e-06,-0.12648,2,1,0
7,LP001056,0,1,0,0,-0.0188307,-2.57765e-06,0.000872078,2,0,2
8,LP001059,0,1,1,0,0.101788,-2.57765e-06,0.193347,1,1,1
9,LP001067,0,0,0,0,-0.0371485,-0.0124633,-0.0338602,2,1,0


In [15]:
prediction = random_forest.predict(test[:,1:])

In [16]:
arr = np.zeros([367,2], dtype='object')
arr[0]

for i in range(367):
    arr[i,0] = test[i,0]
    if prediction[i] == 0:
        arr[i,1] = 'N'
    else:
        arr[i,1] = 'Y'

In [17]:
df = pd.DataFrame(arr)

In [18]:
df.to_csv('results.csv')

In [19]:
pd.DataFrame(arr[0:10])

,0,1
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,N
4,LP001051,Y
5,LP001054,Y
6,LP001055,Y
7,LP001056,N
8,LP001059,Y
9,LP001067,Y
